In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, sys

import py4DSTEM

In [ ]:


# Load original Nion Swift data files or py4DSTEM .h5 files.
def load_data(data_path):

    import copy
    from pathlib import Path

    from py4DSTEM.io import (
        save,
        read
    )
    
    import json
    import h5py

    filetype = str(os.path.splitext(data_path)[1])
    
    if filetype == '.ndata1' or filetype == '.ndata':
        data = np.load(data_path)
        metadata = json.loads(data['metadata.json'])

        return data['data'], metadata
    
    elif filetype == '.h5':
        try:
            data = read(data_path, data_id=0)
            return data, None
        except:
            file = h5py.File(data_path, 'r')
            data = file['data'][:]
            metadata = json.loads(file['data'].attrs['properties'])
            file.close()
            return data, metadata
        



# Iterative differential phase contrast reconstruction of the phase, equivalent to iCOM.
# If a path is given for `write_dc`, adds the Q-R rotation angle into the DataCube file on disk.

def return_icom(dc, me, intermediate_outputs=True, plot_output=True, return_object=False, write_dc=''):
    
    from py4DSTEM.process.phase import DPC

    try:
        E = me['metadata']['instrument']['high_tension']
    except KeyError:
        E = me['energy']
    
    dpc = DPC(
        dc,
        device='cpu',
        energy=E,
        verbose=intermediate_outputs,
    ).preprocess(
        maximize_divergence=False,
        figsize=(6,6),
        cmap='PiYG',
        plot_rotation = True,
        plot_center_of_mass = True,
        #progress_bar=intermediate_outputs,
    )
    
    # DPC CoM signals are typically noisy, and a much more reliable angle is detected when blurring.
    from scipy.ndimage import gaussian_filter

    blurradius=5

    if intermediate_outputs:
        print("Applying a Gaussian blur of " + str(blurradius) + " px to COM signals and redoing.")
    
    com_x_filtered = gaussian_filter(dpc._com_measured_x, blurradius)
    com_y_filtered = gaussian_filter(dpc._com_measured_y, blurradius)
    
    dpc = DPC(
        dc,
        device='cpu',
        energy=E,
        verbose=intermediate_outputs,
    ).preprocess(
        maximize_divergence=False,
        figsize=(6,6),
        cmap='PiYG',
        force_com_measured = (com_x_filtered, com_y_filtered),
        plot_rotation = intermediate_outputs,
        plot_center_of_mass = intermediate_outputs,
        #progress_bar=intermediate_outputs,
    )
    
    rotation = dpc._rotation_best_rad * 180 / np.pi
    dc.calibration.set_QR_rotation_degrees(rotation)
    
    if write_dc != '':
        py4DSTEM.save(write_dc + '.h5', dc, 'o')
        
    icom = dpc.reconstruct(
        progress_bar=intermediate_outputs,
        store_iterations=False,
        max_iter=20,
        reset=True,
        # q_highpass = 0.6,
    )

    if intermediate_outputs:
        dpc.visualize(cbar=True)

    if plot_output:
        
        fig, ax = plt.subplots()
        try:
            fov = me['metadata']['scan']['fov_nm'] * 10
            im1 = ax.imshow(dpc.object_phase, extent=(0, fov, 0, fov), interpolation='none')
        except KeyError:
            fov = me['fov']
            im1 = ax.imshow(dpc.object_phase, extent=(0, fov[0], 0, fov[1]), interpolation='none')
        
        ax.set_title('iCoM phase (py4DSTEM)')
        ax.set_xlabel('x (Å)')
        ax.set_ylabel('y (Å)')
        fig.colorbar(im1,ax=ax)
        plt.show(fig, block=False)
    
    if return_object:
        return icom
    else:
        return icom.object_phase * 2*np.pi




In [ ]:
from py4DSTEM import DataCube
file_path ="/home/somar/Nionswift sessions Somar/2025/07-2025/30-07-2025_id2525_checking_after_laser_heating_20percent_10min/Nion Swift Project 20250730 Data/2025/07/30/20250730-183052/data_DN31P0573EG8SBUG0AFSSXCTF.h5"
fp2 = "/home/somar/Nionswift sessions Somar/2025/07-2025/30-07-2025_id2525_checking_after_laser_heating_20percent_10min/Nion Swift Project 20250730 Data/2025/07/30/20250730-183052/data_3TEB3B0YE3MWRGP5K2UUJ0CXO.h5"
fp3 = "/home/somar/Nionswift sessions Somar/2025/07-2025/10-07-2025_checking_after_heating_150C_3h_sample_2525/Nion Swift Project 20250710 Data/2025/07/10/20250710-094134/data_VGFYKQAK44X7QJV6B15F6KS5D.h5"
fp4 = "/home/somar/Nionswift sessions Somar/2025/07-2025/10-07-2025_checking_after_heating_150C_3h_sample_2525/Nion Swift Project 20250710 Data/2025/07/10/20250710-094134/data_BYB5BTQE019T7TJH267ZL5A7N.h5"
fp5 = "/home/somar/Nionswift sessions Somar/2025/07-2025/10-07-2025_checking_after_heating_150C_3h_sample_2525/Nion Swift Project 20250710 Data/2025/07/10/20250710-111019/data_WBTVOQX752WHGQM5VD90NBO1E.h5"
data, metadata = load_data(file_path)
return_icom(
    DataCube(data),
    metadata,
    intermediate_outputs=True,
    plot_output=True,
    write_dc='')
print(data.shape)
print(data.T.shape)


In [ ]:
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from skimage import exposure
import cv2
fp1 = '/home/somar/testing_rotation_transpose/Spectrum Image HAADF 32.ndata1'
fp2 = '/home/somar/testing_rotation_transpose/SSB (AC) of Spectrum Image Ronchigram 32.ndata1'
fp3 = '/home/somar/testing_rotation_transpose/SSB (AC) of Spectrum Image Ronchigram 16.ndata1'
fp4 = '/home/somar/testing_rotation_transpose/SSB (AC) of Spectrum Image Ronchigram 31.ndata1'
def load_ndata1_image(path):
    """
    Load an image from a .ndata1 file, assuming it's HDF5 with one dataset.
    Returns a 2D NumPy array.
    """
    if not os.path.isfile(path):
        raise FileNotFoundError(f"File {path} not found.")
    data = np.load(path, allow_pickle=True)
    if 'data' in data:
        return data['data']
    elif 'image' in data:
        return data['image']
    else:
        raise ValueError(f"File {path} does not contain expected data structure. Ensure it is a valid .ndata1 file.")
    

def enhance_contrast_brightness(image, clip_percent=0.3, gamma=2.0):
    """
    Improve image contrast and brightness using contrast stretching and gamma correction.
    - clip_percent: percentage of histogram to clip from both ends (0–100).
    - gamma: <1 = brighter, >1 = darker.
    """
    # Clip histogram extremes
    p_low, p_high = np.percentile(image, (clip_percent, 100 - clip_percent))
    image_rescaled = exposure.rescale_intensity(image, in_range=(p_low, p_high))

    # Ensure image is strictly non-negative for gamma correction
    image_rescaled = np.clip(image_rescaled, 0 , None)

    # Gamma correction
    image_gamma = exposure.adjust_gamma(image_rescaled, gamma=gamma)

    return image_gamma

    

def rotate_image(image_path, angle_deg,
                 clip_percent=1, gamma=1.0, transpose=False):
    image = load_ndata1_image(image_path)
    rotated = rotate(image, angle=angle_deg, reshape=False, order=3, mode='reflect')
    
    image_enh = enhance_contrast_brightness(image, clip_percent, gamma)
    if transpose:
        rotated = np.transpose(rotated)
    rotated_enh = enhance_contrast_brightness(rotated, clip_percent, gamma)
    
    return rotated, rotated

def show_original_vs_rotated(original, rotated, angle_deg):
    """Show original and rotated images side by side."""
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(original, cmap='gray')
    axs[0].set_title("Original Image")
    axs[0].axis('off')

    axs[1].imshow(rotated, cmap='gray')
    axs[1].set_title(f"Rotated by {angle_deg}°")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()
angle_deg = 0 # Example rotation angle
original_image = load_ndata1_image(fp4)

import numpy as np

a = np.max(original_image) + np.min(original_image)
recovered = a - original_image

_, rotated_image = rotate_image(fp4, angle_deg= angle_deg, clip_percent=0.4, gamma=3, transpose=True)

show_original_vs_rotated(recovered, rotated_image, angle_deg)